In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import seaborn as sb
import calendar
import datetime as dt

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

from pylab import rcParams
rcParams['figure.figsize'] = 35, 40

import fnmatch
import os
from tqdm import tqdm

## Читаем данные

In [2]:
data=pd.read_pickle('MG_Sales.pickle',compression='gzip')

In [3]:
#получаем колонки категориальных характеристик
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']

In [4]:
#заполняем пустые значения в категориальных данных
data[categorical_columns]=data[categorical_columns].applymap(lambda cell: cell if bool(cell.strip()) else '<Неопределено>')

## Визуализируем данные

In [79]:
def visualize_data(data,title,col,val,lim=-1):
    if lim>0:
        title+=' '+str(lim)+' максимальных позиций'

    data_table=data.pivot_table(index=['Год','Месяц','День'], columns=col, values=val, aggfunc=sum,fill_value=0,dropna=False)

    cols=data.groupby(col)[val].sum()
    cols.sort_values(ascending=False,inplace=True)
    
    
    maxval=0
    
    def convertData(ymd):
        y=ymd.name[0]
        m=ymd.name[1]
        d=ymd.name[2]
        try: 
            if (m==2)&(d==29): 
                return dt.datetime(1980,1,1)
            else:
                return dt.datetime(y,m,d) 
        except ValueError: 
            return dt.datetime(1980,1,1)

    #преобразуем дату
    data_table['Дата']=data_table.apply(convertData, axis=1)

    #добавляем колонки с датами
    jobDay=data[data['РабочиеВыходные']>2].groupby(['Год','Месяц','День'])['РабочиеВыходные'].max()
    celebrate=data[data['Праздник']==1].groupby(['Год','Месяц','День'])['Праздник'].max()

    #соединяем все
    data_table=pd.concat([data_table, jobDay,celebrate], axis=1).reset_index()#, join='inner'

    #удаляем лишнее
    data_table.drop(['Год','Месяц','День'], axis=1, inplace=True)
    data_table.drop(data_table[data_table['Дата']==dt.datetime(1980,1,1)].index, inplace=True)
    data_table.drop(data_table[data_table['Дата']>=dt.datetime.now()].index, inplace=True)
    
    df=data_table.set_index('Дата')
        
    init_notebook_mode(connected = True)
    _data = []    
    for column in cols.index:
        if lim==0: break
        lim-=1            
        
        _max=df[column].max()
        if _max>maxval: maxval=_max
        
        
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',            
            name = column,
            line=dict(
                shape='spline'
            )
        )                        
        _data.append(trace)
        
    for column,clr in zip(['РабочиеВыходные','Праздник'],['rgba(200, 200, 16, 0.5)','rgba(250, 100, 16, 0.5)']):
        trace = go.Bar(
            x = df.index,
            y = df[column].map(lambda val: maxval if val>0 else 0),
            name = column,
            opacity=0.6,
            marker=dict(
                color=clr,
                    line=dict(
                        color=clr,
                        width=0,
                    )
                )   
            )
        _data.append(trace)
    
    layout = dict(title = title)
    
    fig = dict(data = _data, layout = layout)
    iplot(fig, show_link=False)

In [81]:
visualize_data(data,'Продажи','ЦветМеталла','Количество',5)
